In [14]:
# Gọi các thư viện cần thiết 
import numpy as np
import pandas as pd # Xu lý bảng
import seaborn as sns # Vẽ biểu đồ thị của dữ liệu
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler # Xử lý chuẩn hóa dữ liệu
from sklearn.model_selection import train_test_split # Chia dữ liệu ra làm 2 phần
from keras.layers import Dense, Activation, Dropout, BatchNormalization, LSTM    # LSTM  biên dạng ANN, BatchNormalization: cho nhỏ lại
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical # Sử dung để làm nổi đối tượng cần phân loại
from keras import callbacks 
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score # Để đo lường

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os



In [15]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_dataset = image_generator.flow_from_directory(batch_size=32,directory='../input/qhuyver2/huyver1',shuffle=True,target_size=(32, 32), subset="training", class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=32,directory='../input/qhuyver2/huyver1',shuffle=True,target_size=(32, 32), subset="validation",class_mode='categorical')

In [16]:
import glob
Huy = glob.glob('../input/qhuyver2/huyver1/Huy/*.*')
NoHuy = glob.glob('../input/qhuyver2/huyver1/NoHuy/*.*')

data = []
labels = []

for i in Huy:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (32,32))
    image=np.array(image)
    data.append(image)
    labels.append(0)
for j in NoHuy:   
    image=tf.keras.preprocessing.image.load_img(j, color_mode='rgb', target_size= (32,32))
    image=np.array(image)
    data.append(image)
    labels.append(0)


data = np.array(data)
labels = np.array(labels)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [18]:
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [19]:
x_train = X_train.reshape(103,3072)
x_test = X_test.reshape(26,3072)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train/=255
x_test/=255
y_train = np_utils.to_categorical(y_train,2)
y_test = np_utils.to_categorical(y_test,2)

In [20]:

# Tạo mảng neron nhân tạo

model = Sequential()
model.add(Dense(512,activation = 'relu',input_shape=(3072,)))
model.add(Dropout(0.25))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(500,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(2))
#model.add(Dense(1))
model.summary()

In [21]:
# Huận luyện mô hình 
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics = ['accuracy']) 

In [22]:
# Train model
history = model.fit(x_train,y_train,batch_size=64,epochs=100,verbose=1,validation_data=(x_test,y_test),callbacks=[EarlyStopping(monitor='val_loss',patience=20)])

In [23]:
# Đánh giá độ chính xác của mô hình
score = model.evaluate(x_test,y_test,verbose=0)
print('Sai số kiểm tra là: ',score[0])
print('Độ chính xác kiểm tra là: ',score[1])

In [24]:
# Vẽ lại quá trình học
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train','Validation'])
plt.show()

In [25]:
from tensorflow.keras.models import load_model
model.save('huyface.h5')
model = load_model('huyface.h5')

In [26]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
filename = "../input/qhuyver2/huyver1/NoHuy/nohuy (1).jpg"

predict = ["This is Huy","This isn't Huy"]
predict = np.array(predict)

img = load_img(filename,target_size=(32,32))
img = img_to_array(img)
img = img.reshape(1,32*32*3)
img = img.astype('float32')
img = img/255

result = np.argmax(model.predict(img),axis=-1)
predict[result]